## Example: "AMR102: Advection of Particles Around Obstacles"

### What Features Are We Using

* Mesh data with EB 
* Linear solvers (multigrid)
* Particle-Mesh interpolation

### The Problem

Recall our previous problem of the drop of dye in a thin incompressible fluid that is spinning 
clock-wise then counter-clockwise with a prescribed motion.  

Now instead of advecting the dye as a scalar quantity defined on the mesh (the
continuum representation), we define the dye as a collection of particles that
are advected by the fluid velocity.

Again the fluid is thin enough that we can model this as two-dimensional
motion; again we have the option of solving in a 2D or 3D computational domain.

To make things even more interesting, there is now an object in the flow, in this case a cylinder.
It would be very difficult to analytically specify the flow field around the object, so instead 
we project the velocity field so that the resulting field represents incompressible flow around the object.

### Projecting the Velocity for Incompressible Flow around the Cylinder

Mathematically, projecting the specified velocity field means solving  

$$\nabla \cdot (\beta \nabla \xi)  = \nabla \cdot \bf{u^{spec}}$$

and setting 

$$\bf{u} = \bf{u^{spec}} - \beta \nabla \xi$$

To solve this variable coefficient Poisson equation, we use the native AMReX geometric multigrid solver.
In our case $\beta = 1 / \rho = 1$ since we are assuming constant density $\rho.$

Note that for this example we are solving everything at a single level for convenience,
but linear solvers, EB and particles all have full multi-level functionality.

In each timestep we compute the projected velocity field, advect the particles with this velocity, then interpolate the particles onto the mesh to determine $\phi(x,y,z)$.

### Particle-In-Cell Algorithm for Advecting $\phi$

We achieve conservation by interpreting the scalar $\phi$ as the number
density of physical dye particles in the fluid, and we represent these physical
particles by "computational" particles $p$. Each particle $p$ stores a
weight $w_p$ equal to the number of physical dye particles it represents.

This allows us to define particle-mesh interpolation that converts
$\phi(x,y,z)$ to a set of particles with weights $w_p$. First, we
interpolate $\phi(x,y,z)$ to $\phi_p$ at each particle location by
calculating:

$$\phi_p = \sum_i \sum_j \sum_k S_x \cdot S_y \cdot S_z \cdot \phi(i,j,k)$$

where in the above, $S_{x,y,z}$ are called shape factors, determined by the
particle-in-cell interpolation scheme we wish to use.

The simplest interpolation is nearest grid point (NGP), where $S_{x,y,z} = 1$
if the particle $p$ is within cell $(i,j,k)$ and $S_{x,y,z} = 0$
otherwise. An alternative is linear interpolation called cloud-in-cell (CIC),
where the shape factors are determined by volume-weighting the particle's
contribution to/from the nearest 8 cells (in 3D).

Once we have interpolated $\phi(x,y,z)$ to the particle to get $\phi_p$, we
scale it by the volume per particle to get the number of physical dye particles
that our computational particle represents. Here, $n_{ppc}$ is the number of
computational particles per cell.

$$w_p = \phi_p \cdot \dfrac{dx dy dz}{n_{ppc}}$$

To go back from the particle representation to the grid representation, we
reverse this procedure by summing up the number of physical dye particles in
each grid cell and dividing by the grid cell volume. This recovers the number
density $\phi(x,y,z)$.

$$\phi(i,j,k) = \dfrac{1}{dx dy dz} \cdot \sum_p S_x \cdot S_y \cdot S_z \cdot w_p$$

This approach is the basis for Particle-In-Cell (PIC) methods in a variety of
fields, and in this tutorial you can experiment with the number of particles
per cell and interpolation scheme to see how well you can resolve the dye
advection.

## Running the code

Similar to the last example, the following parameters can be set at run-time -- these are currently set in the inputs file.

```
stop_time =  2.0                         # the final time (if we have not exceeded number of steps)
max_step  = 200                          # the maximum number of steps (if we have not exceeded stop_time)

n_cell = 64                              # number of cells in x- and y-directions; z-dir has 1/8 n_cell (if 3D)

max_grid_size = 32                       # the maximum number of cells in any direction in a single grid

plot_int = 10                            # frequency of writing plotfiles

```

The size, orientation and location of the cylinder are specified in the inputs file as well:

```
cylinder.direction = 2                  # cylinder axis aligns with z-axis
cylinder.radius    = 0.1                # cylinder radius
cylinder.center    = 0.7 0.5 0.5        # location of cylinder center (in domain that is unit box in xy plane)

cylinder.internal_flow = false          # we are computing flow around the cylinder, not inside it
```

Here you can play around with changing the size and location of the cylinder.

The number of particles per cell and particle-mesh interpolation type are also specified in the inputs file:

```
n_ppc = 100                              # number of particles per cell for representing the fluid

pic_interpolation = 1                    # Particle In Cell interpolation scheme:
                                         # 0 = Nearest Grid Point
                                         # 1 = Cloud In Cell
```

You can vary the number of particles per cell and interpolation to see how they influence the smoothness of the phi field.

## Visualization

In [ ]:
import yt
from yt.frontends.boxlib.data_structures import AMReXDataset

In [ ]:
ds = AMReXDataset("plt00120")

In [ ]:
ds.field_list

In [ ]:
sl = yt.SlicePlot(ds, 2, ('boxlib', 'phi'))
sl.annotate_particles(ds.domain_width[2], stride=10, marker='.')